In [12]:
import pandas as pd
import sys
import logging
import json
sys.path.append('../')
from llm_api_client import LLMAPIClient

In [2]:
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

logger = logging.getLogger(__name__)

In [3]:
prompt = """다음은 학생이 제출한 수행평가 과제물이다. 제시된 평가 기준을 참고하여 과제물을 체계적으로 분석하고 평가하라. 평가 시 다음 단계를 순차적으로 따르라:

1. 평가기준을 꼼꼼히 읽고 이해하라.
2. 학생의 제출물을 세심히 분석하며, 평가 기준에 부합하거나 미흡한 점이 있는지 확인하라. 만약 직접적으로 인용할 수 있는 표현이 있다면, 해당 표현을 정확히 인용하라.
3. 인용한 표현이 있다면 평가 기준과의 관계를 논리적으로 설명하라.
4. 평가 항목별로 정량적 점수를 부여하며 근거를 구체적으로 기술하라.

학생 제출물:
[학생 제출물]"""

system_instruction ="""너는 고등학교 [과목] 담당 교사이며, 학생이 제출한 [수행평가 제목] 수행평가 과제물을 객관적이고 공정하게 평가하는 전문 평가자 역할을 수행한다. 다음 지침을 명확히 숙지하여 평가하라.
수행평가에 대한 세부정보는 다음과 같다:

학년: [학년]
수행평가 제목: [수행평가 제목]
수행평가 유형: [수행평가 유형]
수행평가 설명: [수행평가 설명]
평가 기준:
[평가 기준]

평가 수행 지침
1. 평가 기준 엄격 준수
- 제공된 평가 기준 및 체크리스트의 항목을 정확히 준수하여 평가하라.

2. 직접적 인용 및 명확한 설명
- 학생의 제출물에서 인용된 표현을 근거로 사용한다면 해당 표현이 평가 기준에 적합하거나 미흡한 이유를 구체적으로 설명하라.

3. 논리적이고 객관적인 근거
- 평가 결과가 일관되도록 하고, 점수 부여의 근거를 명확히 기술하여 객관성과 신뢰성을 높이도록 하라.

결과는 JSON 객체로만 출력하라. 마크다운 코드 펜스(```)는 포함하지 말고, 모든 문자열이 올바르게 따옴표로 감싸지도록 하라."""

output_schema = {
    "type": "object",
    "properties": {
        "checklist": {
            "type": "array",
            "description": "평가 항목별 점수와 이유",
            "items": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "평가 대분류"
                    },
                    "subcategories": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "name": {
                                    "type": "string",
                                    "description": "평가 소분류"
                                },
                                "score": {
                                    "type": "integer",
                                    "description": "0~3 사이의 점수"
                                },
                                "reason": {
                                    "type": "string",
                                    "description": "점수 평가 이유"
                                },
                                "evidence":{
                                    "type": "array",
                                    "description": "관련 있는 텍스트를 증거로 제시",
                                    "items": {
                                        "type": "string",
                                        "description": "증거 텍스트"
                                    }
                                }
                            },
                            "required": ["name", "score", "reason", "evidence"]
                        }
                    }
                },
                "required": ["title", "subcategories"]
            }
        },
        "overall_feedback": {
            "type": "string",
            "description": "전체적인 피드백"
        }
    },
    "required": ["checklist", "overall_feedback"]
}

In [4]:
df_aihub = pd.read_csv('C:/Users/yuhan/Desktop/CSD_18기/베어러블/AI_Hub/에세이_글_평가_데이터_전처리완.csv')

essay_dict = df_aihub.iloc[3000].to_dict()

In [5]:
with open('../checklists/논술형_checklist.json', 'r', encoding='utf-8') as f:
    checklist = json.load(f)

def format_template(template, values):
    """Format a template with the given values
    
    Args:
        template: The template string with [placeholders]
        values: Dictionary of placeholder values
        
    Returns:
        Formatted template with placeholders replaced
    """
    formatted = template
    replacements = {
        '학년': values.get('student_grade', ''),
        '과목': values.get('subject', '국어'),
        '수행평가 제목': values.get('essay_main_subject', ''),
        '수행평가 유형': values.get('essay_type', ''),
        '수행평가 설명': values.get('essay_prompt', ''),
        '평가 기준': values.get('checklist', json.dumps(checklist, ensure_ascii=False, indent=2)),
        '학생 제출물': values.get('processed_text', '')
    }
    
    for placeholder, value in replacements.items():
        formatted = formatted.replace(f"[{placeholder}]", value)
        
    return formatted

formatted_prompt = format_template(prompt, essay_dict)
formatted_system_instruction = format_template(system_instruction, essay_dict)

In [6]:
# Initialize the client
client = LLMAPIClient()
client.logger = logger

2025-06-21 11:18:43,025 - INFO - Logging initialized. Log file: logs\api_interactions_20250621_111843.log
2025-06-21 11:18:43,662 - INFO - LLM API Client initialized


In [7]:
models_list = [
    "gemini-2.5-flash",
    "gemini-2.5-flash-lite-preview-06-17",
    "gemini-2.0-flash",
    "gemini-2.0-flash-lite",
    "claude-sonnet-4-20250514",
    "claude-3-7-sonnet-20250219",
    "claude-3-5-haiku-20241022",
    "claude-3-5-sonnet-20241022",
    "claude-3-haiku-20240307",
    "gpt-4.1",
    "gpt-4.1-mini",
    "gpt-4.1-nano",
    "gpt-4o",
    "gpt-4o-mini",
    "o4-mini",
    "o3-mini"
]

In [8]:
params = {
    "prompt": formatted_prompt,
    "system_instruction": formatted_system_instruction,
    "models": models_list,
    "response_schema": output_schema,
    "file_path": None,
    "temperature": 0.1,
    "max_tokens": 4096,
    "enable_thinking": False
}

results = await client.generate_responses(**params)

2025-06-21 11:18:43,703 - INFO - Processing with Gemini model: gemini-2.5-flash
2025-06-21 11:18:43,706 - INFO - Gemini 모델에 요청 전송 중: gemini-2.5-flash
2025-06-21 11:18:43,706 - INFO - Processing with Gemini model: gemini-2.5-flash-lite-preview-06-17
2025-06-21 11:18:43,706 - INFO - Processing with Gemini model: gemini-2.0-flash
2025-06-21 11:18:43,707 - INFO - AFC is enabled with max remote calls: 10.
2025-06-21 11:18:43,707 - INFO - Processing with Gemini model: gemini-2.0-flash-lite
2025-06-21 11:18:43,709 - INFO - Processing with Anthropic model: claude-sonnet-4-20250514
2025-06-21 11:18:43,711 - INFO - Processing with Anthropic model: claude-3-7-sonnet-20250219
2025-06-21 11:18:43,711 - INFO - Gemini 모델에 요청 전송 중: gemini-2.5-flash-lite-preview-06-17
2025-06-21 11:18:43,712 - INFO - Processing with Anthropic model: claude-3-5-haiku-20241022
2025-06-21 11:18:43,714 - INFO - Gemini 모델에 요청 전송 중: gemini-2.0-flash
2025-06-21 11:18:43,714 - INFO - Processing with Anthropic model: claude-3-5

In [13]:
client.save_results(results, file_path=f"{essay_dict['essay_id']}_evaluation_results.json")

# filter out models that failed to generate a response
filtered_results = {model: result for model, result in results.items() if 'response' in result and model not in ['gemini-2.5-flash-lite-preview-06-17', 'o4-mini']}

processed_results = client.process_results(filtered_results)

scores_df = client.extract_scores(filtered_results, essay_id=essay_dict['essay_id'])
scores_df.insert(loc=1, column='trial', value=1)

2025-06-21 11:28:04,031 - INFO - Results saved to ESSAY_74093_evaluation_results.json
2025-06-21 11:28:04,033 - WARNING - claude-3-5-haiku-20241022의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.
2025-06-21 11:28:04,035 - WARNING - claude-3-5-haiku-20241022의 응답이 유효한 JSON이 아닙니다. 원시 텍스트를 반환합니다.


In [10]:
scores_df

,essay_id,trial,model,response_time,문법의 정확성,단어 사용의 적절성,문장 표현의 적절성,문단 내 구조의 적절성,문단 간 구조의 적절성,구조의 일관성,분량의 적절성,주제의 명료성,설명의 구체성,사고의 창의성,프롬프트 독해
0,ESSAY_74093,1,gemini-2.5-flash,10.400217,1,1,1,2,1,1,3,2,2,2,3
1,ESSAY_74093,1,gemini-2.0-flash,9.241234,2,2,2,2,2,2,3,2,2,1,3
2,ESSAY_74093,1,gemini-2.0-flash-lite,9.702807,2,2,2,2,1,1,2,2,1,2,2
3,ESSAY_74093,1,claude-sonnet-4-20250514,44.893014,1,2,1,2,1,2,3,2,2,2,3
4,ESSAY_74093,1,claude-3-7-sonnet-20250219,41.109349,1,2,1,1,1,1,2,2,1,1,2
5,ESSAY_74093,1,claude-3-5-sonnet-20241022,24.409353,1,2,2,2,2,2,3,3,2,2,3
6,ESSAY_74093,1,claude-3-haiku-20240307,14.423131,3,3,3,3,3,3,3,3,3,3,3
7,ESSAY_74093,1,gpt-4.1,14.990704,2,2,2,2,2,2,3,3,2,2,3
8,ESSAY_74093,1,gpt-4.1-mini,21.583748,2,2,2,2,2,2,3,3,2,1,3
9,ESSAY_74093,1,gpt-4.1-nano,7.370524,2,2,2,2,2,2,2,3,2,2,3


In [15]:
essay_dict

{'student_grade': '고등_2학년',
 'essay_id': 'ESSAY_74093',
 'essay_type': '대안제시',
 'essay_main_subject': '장애에 대한 부정적인 인식 개선방안',
 'essay_prompt': " 선천적으로 장애를 가지고 태어난 사람들도 존재하지만, 대부분 장애를 가진 사람은 후천적인 사고를 이유로 장애를 얻어 살아가는 사람들입니다.\r\n\r\n 즉 사고 때문에 손가락을 절단당하거나 다리를 잃는다거나 하는 경우도 모두 후천적인 사고로 인한 장애라고 할 수 있습니다.\r\n \r\n 나라에서는 장애를 가진 사람들을 위한 일자리나 사회의 일원으로서 당당하게 살아갈 수 있도록 인식개선교육 및 여러 일자리를 신설해서 생활에 지장이 없도록 돕고 있습니다. 사회적으로도 '장애인'보다는 생활하기에 몸이 조금 불편한 사람으로 인식이 조금씩 변해가고 있습니다. \r\n\r\n 하지만 사회적 인식이 변해간다고 해도 기본적으로 모든 사람의 생각까지는 바꾸지 못합니다. 또한, 사회적인 혜택을 통해 장애인들에 대한 복지를 개선하는 방향이 무조건 그들에게 도움이 되지는 않을 것입니다.\r\n\r\n 여러분들이 생각하기에 장애를 가진 사람들에 대한 인식을 바꿀 수 있는 가장 좋은 방법은 무엇이라고 생각하나요? \r\n\r\n 학교나 직장 내에서 장애인에 대한 인식을 개선하는 방법을 생각해보고 작성해주세요. 또한, 그들이 사회에 적응할 수 있는 방안에 대해 서술해주세요.",
 'processed_text': '장애 발생 원인 중 후천적 원인이 88% 이상으로 선천적 원인보다 후천적 원인이 더 크다. 다시 말해서, 어느 누구도 장애인이 될 가능성에서 자유롭지는 않다. 우리가 당연하다고 생각하는 것들을 장애인의 입장이 되어 생각해 보는 장애감수성이 필요하다.학교내에서 장애인학생에 대한 인식을 개선하기 위한 방안을 생각해 보았다.첫째, 장애에 대한 오해와 무지는 부정적 태도를 형성하므로 유치원때부터 장애인 인식개선 의무교육을 시행한

In [16]:
processed_results

{'gemini-2.5-flash': {'text': {'checklist': [{'title': '표현',
     'subcategories': [{'name': '문법의 정확성',
       'score': 1,
       'reason': '문법적 오류가 다소 많습니다. 특히 문장 연결과 조사의 사용에서 오류가 자주 발견됩니다.',
       'evidence': ['일방적 배려만 비장애 학생에게 강요하지 않고 학생들의 입장을 충분히 들어주어야 행동 변화가 올 것이다.',
        '생김새와 말투가 달라서 그냥 왠지 불편함을 이유로 장애에 대한 잘못된 인식과 편견, 고정관념들을 교육을 통해서 알아간다면 장애에 대한 인식이 바뀌게 될 것이다.',
        '장애를 지닌 사람들은 자신들의 의지로 장애를 가지고 싶어서 그러하게 된 것도 아니었을 것인데 그 분들을 향해 부정적인 의식을 지니게 되었는지 생각을 해보며 지금까지 나 자신을 돌아보며 장애인이라는 사람들의 생각을 다시 성찰해보자.']},
      {'name': '단어 사용의 적절성',
       'score': 1,
       'reason': "상황에 맞지 않는 단어 사용이 꽤 보이며, 단어의 사용이 다양하지 못합니다. 예를 들어 '올렸다'와 같은 부적절한 표현은 없으나, '긍정적으로 개선해야 한다'와 같이 어색한 표현이 있습니다.",
       'evidence': ['장애인 학생에 대한 일방적 이해를 강요하는 부정적 인식을 긍정적으로 개선해야 한다.',
        '장애인이라는 사람들의 생각을 다시 성찰해보자.']},
      {'name': '문장 표현의 적절성',
       'score': 1,
       'reason': '문장 구조의 다양성이 다소 부족하며, 문장의 길이가 길고 복잡하여 의미 전달이 명확하지 않은 경우가 종종 나타납니다.',
       'evidence': ['생김새와 말투가 달라서 그냥 왠지 불편함을 이유로 장애에 대한 잘못된 인식과 편